In [1]:
import pandas as pd
import wandb
import keras
from wandb.keras import WandbCallback
import shutil
import tensorflow as tf
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


#### Setup network architecture

In [2]:
# lr: float >= 0. Learning rate.
# beta_1: float, 0 < beta < 1. Generally close to 1.
# beta_2: float, 0 < beta < 1. Generally close to 1.
# epsilon: float >= 0. Fuzz factor.
# decay: float >= 0. Learning rate decay over each update.
##Default params
# lr=0.001,
# beta_1=0.9,
# beta_2=0.999,
# epsilon=1e-08,
# decay=0.0,

hyper_params = dict(
    image_size = 224,
    num_layers_frozen = 4 + 5*6,
    num_layers_pop = 6*6,
    dense_1_size = 1024,
    dense_2_size = 512,
    dense_3_size = 128,
    batch_size = 32,
#     steps_per_epoch = 200,
    #Adam Optimizer
    lr=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    decay=0.000,
    epochs = 5
)

wandb.init(project="car_nanodegree_capstone", config=hyper_params)

config = wandb.config

dataBaseFolder = '../../../../data/'
# datasdcnd = dataBaseFolder + 'dataset-sdcnd-capstone/data/real_training_data/'
dataTL = dataBaseFolder + 'tl_engineer5/'

base_model = keras.applications.mobilenet.MobileNet(input_shape=(config.image_size,config.image_size,3), 
                                                    include_top=False, weights='imagenet')

for layer in base_model.layers[:config.num_layers_frozen]:
    layer.trainable=False

for layer in base_model.layers[config.num_layers_frozen:]:
    layer.trainable=True

x = base_model.layers[-config.num_layers_pop-1].output
x = GlobalAveragePooling2D()(x)
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(config.dense_1_size,activation='relu')(x) 
x = Dense(config.dense_2_size,activation='relu')(x) #dense layer 2
x = Dense(config.dense_3_size,activation='relu')(x) #dense layer 3
preds = Dense(4,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input, outputs=preds)

adam_opt = Adam(lr=config.lr, beta_1=config.beta_1, beta_2=config.beta_2, epsilon=config.epsilon, decay=config.decay)

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ········


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    dataTL+'test',
    target_size=(config.image_size, config.image_size),
    batch_size=1,
)

train_generator = train_datagen.flow_from_directory(
        dataTL+'train',
        target_size=(config.image_size, config.image_size),
        batch_size=config.batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        dataTL+'val',
        target_size=(config.image_size, config.image_size),
        batch_size=config.batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        epochs=config.epochs,
        steps_per_epoch=3521 // config.batch_size,
        validation_data=validation_generator,
        validation_steps = 755//config.batch_size,
        callbacks=[WandbCallback()])


model.save(os.path.join(wandb.run.dir, "model_34_frozen_36_pop.h5"))

saver = tf.train.Saver()
saver.save(K.get_session(), '/tmp/keras_model.ckpt')

Found 757 images belonging to 4 classes.
Found 3521 images belonging to 4 classes.
Found 755 images belonging to 4 classes.
Epoch 1/5
110/110 [==============================] - 180s - loss: 0.8867 - acc: 0.6540 - val_loss: 1.1029 - val_acc: 0.5163
Epoch 2/5
110/110 [==============================] - 172s - loss: 0.4643 - acc: 0.8381 - val_loss: 1.2935 - val_acc: 0.5671
Epoch 3/5
110/110 [==============================] - 171s - loss: 0.2947 - acc: 0.9017 - val_loss: 0.1691 - val_acc: 0.9461
Epoch 4/5
110/110 [==============================] - 172s - loss: 0.2591 - acc: 0.9065 - val_loss: 0.2922 - val_acc: 0.8714
Epoch 5/5
110/110 [==============================] - 172s - loss: 0.2115 - acc: 0.9258 - val_loss: 0.1251 - val_acc: 0.9502


'/tmp/keras_model.ckpt'

In [5]:
scores = model.evaluate_generator(generator=test_generator, steps=755//config.batch_size, workers=1)
print('Accuracy: ', scores[1])

filenames = test_generator.filenames
nb_samples = len(filenames)
pred = model.predict_generator(test_generator, steps=nb_samples, verbose=1, workers=1)
predicted_class_indices = np.argmax(pred, axis=1)

#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, predicted_class_indices))

# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]

print('Classification Report')
print(classification_report(test_generator.classes, predicted_class_indices))

# filenames=test_generator.filenames
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})

# results.to_csv(dataBaseFolder + 'results_baseline.csv',index=False)

print(train_generator.class_indices)
print(validation_generator.class_indices)

('Accuracy: ', 0.95652173913043481)
756/757 [============================>.] - ETA: 0sConfusion Matrix
[[ 30  47  47   6]
 [ 48 110 109  24]
 [ 40 119 104  18]
 [  3  26  22   4]]
Classification Report
              precision    recall  f1-score   support

           0       0.25      0.23      0.24       130
           1       0.36      0.38      0.37       291
           2       0.37      0.37      0.37       281
           3       0.08      0.07      0.07        55

   micro avg       0.33      0.33      0.33       757
   macro avg       0.26      0.26      0.26       757
weighted avg       0.33      0.33      0.33       757

{'unknown': 2, 'green': 0, 'yellow': 3, 'red': 1}
{'unknown': 2, 'green': 0, 'yellow': 3, 'red': 1}


In [7]:
import cv2

In [8]:
def predict_img(model, fname):
    cv_image = cv2.imread(fname)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    cv_image = cv2.resize(cv_image, (224, 224))
    cv_image = cv_image/255.
    image_expanded = np.expand_dims(cv_image, axis=0)
    pred = model.predict(image_expanded)
    print(pred)
    return np.argmax(pred, axis=1)

In [9]:
pred_class_arr = []
for img_name in test_generator.filenames:
    pred_class = predict_img(model, os.path.join(dataTL+'test', img_name))
    print(img_name, pred_class)
    pred_class_arr.append(pred_class[0])

[[  9.97914255e-01   3.22367384e-07   4.32107299e-07   2.08492950e-03]]
('green/out09034.png', array([0]))
[[  9.98118699e-01   1.64967403e-07   1.95154570e-07   1.88084098e-03]]
('green/out08316.png', array([0]))
[[  9.91743267e-01   2.92887904e-07   1.77566221e-07   8.25621188e-03]]
('green/out08510.png', array([0]))
[[  9.96641755e-01   9.85558799e-08   7.52571765e-08   3.35801183e-03]]
('green/out01252.png', array([0]))
[[  9.77287054e-01   1.31507068e-06   6.22054415e-07   2.27110125e-02]]
('green/out04896.png', array([0]))
[[  9.93191063e-01   1.56656910e-07   8.39680396e-08   6.80869026e-03]]
('green/out01736.png', array([0]))
[[  9.43463624e-01   3.71298869e-04   2.75760016e-04   5.58892936e-02]]
('green/out08712.png', array([0]))
[[  9.98875082e-01   1.09822382e-08   1.01821840e-08   1.12487678e-03]]
('green/out01866.png', array([0]))
[[  9.92926657e-01   1.18595460e-06   1.03910929e-06   7.07108062e-03]]
('green/out00946.png', array([0]))
[[  9.95181143e-01   6.13435475e-07  

[[  9.98417974e-01   3.98576034e-08   4.10100292e-08   1.58202939e-03]]
('green/out01836.png', array([0]))
[[  9.98052597e-01   2.25274768e-08   1.77346777e-08   1.94741134e-03]]
('green/out02132.png', array([0]))
[[ 0.63166326  0.01221279  0.0071915   0.3489325 ]]
('green/out06208.png', array([0]))
[[  8.99810851e-01   3.67320972e-05   1.30344069e-05   1.00139342e-01]]
('green/out07894.png', array([0]))
[[  9.94429231e-01   2.67382376e-07   1.65141927e-07   5.57041401e-03]]
('green/out08782.png', array([0]))
[[  9.95281398e-01   3.40137888e-07   2.85384886e-07   4.71794745e-03]]
('green/out08740.png', array([0]))
[[  9.94114101e-01   1.77929701e-06   1.92050402e-06   5.88219846e-03]]
('green/out00678.png', array([0]))
[[  9.97642457e-01   9.96642342e-08   9.75324497e-08   2.35729618e-03]]
('green/out01566.png', array([0]))
[[  9.43893492e-01   3.77772027e-04   3.62544408e-04   5.53662516e-02]]
('green/out00042.png', array([0]))
[[  9.97668445e-01   7.95066484e-08   6.17369338e-08   2.

[[  3.45903332e-04   9.91821170e-01   6.09583780e-03   1.73710322e-03]]
('red/out01240.png', array([1]))
[[ 0.03441341  0.84921604  0.0814397   0.03493088]]
('red/out05224.png', array([1]))
[[  3.64351838e-09   9.99993205e-01   2.48141373e-06   4.30056571e-06]]
('red/out04660.png', array([1]))
[[  5.78520348e-06   9.99312043e-01   3.29261820e-04   3.52797593e-04]]
('red/out01916.png', array([1]))
[[  6.30097148e-08   9.99960065e-01   1.73758872e-05   2.25799085e-05]]
('red/out04846.png', array([1]))
[[  2.23517389e-07   9.99873519e-01   3.07132213e-05   9.55014402e-05]]
('red/out04852.png', array([1]))
[[  2.66325600e-07   9.99915838e-01   6.97812720e-05   1.41022210e-05]]
('red/out05972.png', array([1]))
[[ 0.1103208   0.59691012  0.24047364  0.05229547]]
('red/out03318.png', array([1]))
[[  6.79327582e-04   9.86104906e-01   1.02978889e-02   2.91785737e-03]]
('red/out06670.png', array([1]))
[[  1.09020766e-06   9.99769986e-01   1.05895051e-04   1.23040198e-04]]
('red/out03052.png', ar

[[  1.48547615e-05   9.98914599e-01   6.29772316e-04   4.40778735e-04]]
('red/out04204.png', array([1]))
[[  8.87516217e-06   9.99078870e-01   4.17530071e-04   4.94734384e-04]]
('red/out00704.png', array([1]))
[[  1.97728741e-06   9.99703109e-01   2.26661898e-04   6.81638121e-05]]
('red/out07080.png', array([1]))
[[  8.01875212e-05   9.96498704e-01   2.52580061e-03   8.95294303e-04]]
('red/out00116.png', array([1]))
[[  4.75277102e-06   9.99382019e-01   2.45650474e-04   3.67568777e-04]]
('red/out09132.png', array([1]))
[[ 0.05448884  0.45343214  0.47587228  0.01620679]]
('red/out05708.png', array([2]))
[[  6.69111131e-08   9.99966145e-01   2.31655031e-05   1.06023699e-05]]
('red/out06598.png', array([1]))
[[  6.14521341e-05   9.97041523e-01   1.26217608e-03   1.63486321e-03]]
('red/out05250.png', array([1]))
[[  9.30734689e-09   9.99985099e-01   4.20304923e-06   1.07071401e-05]]
('red/out00300.png', array([1]))
[[  4.48861300e-07   9.99877453e-01   9.61487312e-05   2.59709668e-05]]
('r

[[  1.27495126e-08   9.99988317e-01   8.20402056e-06   3.43106467e-06]]
('red/out00650.png', array([1]))
[[ 0.01984387  0.86177826  0.10691833  0.01145952]]
('red/out05728.png', array([1]))
[[  5.73494708e-06   9.99322653e-01   3.20243125e-04   3.51456372e-04]]
('red/out00678.png', array([1]))
[[ 0.04292833  0.30597618  0.64017701  0.0109185 ]]
('red/out05700.png', array([2]))
[[ 0.05086541  0.63194817  0.2960017   0.02118472]]
('red/out07922.png', array([1]))
[[ 0.0423159   0.82301265  0.08760245  0.04706899]]
('red/out05112.png', array([1]))
[[  7.06704974e-04   9.84072983e-01   1.36142392e-02   1.60609558e-03]]
('red/out06194.png', array([1]))
[[  8.96182314e-07   9.99806702e-01   1.08539571e-04   8.38919150e-05]]
('red/out05338.png', array([1]))
[[  5.40419364e-07   9.99810517e-01   5.90618220e-05   1.29868073e-04]]
('red/out01604.png', array([1]))
[[  4.96569601e-06   9.99426365e-01   4.43245808e-04   1.25406834e-04]]
('red/out06656.png', array([1]))
[[  1.14684612e-04   9.9554055

[[  4.90796310e-08   9.99958873e-01   1.30628760e-05   2.80554377e-05]]
('red/out00356.png', array([1]))
[[  3.64351838e-09   9.99993205e-01   2.48141373e-06   4.30056571e-06]]
('red/out04656.png', array([1]))
[[  2.25363465e-06   9.99637365e-01   2.98461033e-04   6.19744242e-05]]
('red/out07388.png', array([1]))
[[  1.43120474e-07   9.99923348e-01   2.42755286e-05   5.21656038e-05]]
('red/out02226.png', array([1]))
[[ 0.13050127  0.51522845  0.29976866  0.05450161]]
('red/out02144.png', array([1]))
[[  3.93220037e-03   1.26536964e-02   9.82987761e-01   4.26290731e-04]]
('unknown/out00762.png', array([2]))
[[  9.13923315e-04   3.30213830e-03   9.95723128e-01   6.08504924e-05]]
('unknown/out03280.png', array([2]))
[[  2.39174871e-04   1.39409059e-03   9.98357594e-01   9.12457199e-06]]
('unknown/out05008.png', array([2]))
[[ 0.10314958  0.36811471  0.49342129  0.03531441]]
('unknown/out07810.png', array([2]))
[[ 0.00869544  0.03415621  0.95601064  0.00113775]]
('unknown/out00428.png', ar

[[  1.59651026e-04   1.78410113e-03   9.98050809e-01   5.44711656e-06]]
('unknown/out05526.png', array([2]))
[[  7.57719623e-04   4.07619495e-03   9.95118380e-01   4.77515787e-05]]
('unknown/out00304.png', array([2]))
[[  2.28611771e-05   3.34196229e-04   9.99642491e-01   3.92427296e-07]]
('unknown/out07682.png', array([2]))
[[  9.16021454e-05   1.19467336e-03   9.98711228e-01   2.57451006e-06]]
('unknown/out00890.png', array([2]))
[[ 0.02173581  0.0656654   0.90868455  0.00391423]]
('unknown/out01542.png', array([2]))
[[ 0.13232291  0.22027558  0.59924841  0.04815307]]
('unknown/out07906.png', array([2]))
[[  3.60051892e-03   1.18883103e-02   9.84133840e-01   3.77448829e-04]]
('unknown/out00728.png', array([2]))
[[  1.25026127e-04   1.75651920e-03   9.98114705e-01   3.73493003e-06]]
('unknown/out03976.png', array([2]))
[[  2.26511783e-03   7.91078899e-03   9.89613831e-01   2.10258222e-04]]
('unknown/out05446.png', array([2]))
[[  6.24044507e-04   3.54855601e-03   9.95792508e-01   3.48

[[ 0.00971454  0.03432808  0.95458061  0.00137677]]
('unknown/out02646.png', array([2]))
[[  1.30584503e-05   1.79806200e-04   9.99806941e-01   1.90666015e-07]]
('unknown/out00910.png', array([2]))
[[ 0.06289564  0.34083349  0.57603061  0.02024037]]
('unknown/out02040.png', array([2]))
[[ 0.01457788  0.14719002  0.83621144  0.00202063]]
('unknown/out05908.png', array([2]))
[[  7.23913079e-03   2.16668341e-02   9.70139503e-01   9.54610179e-04]]
('unknown/out06742.png', array([2]))
[[  2.16283300e-03   2.80829538e-02   9.69573438e-01   1.80739458e-04]]
('unknown/out07474.png', array([2]))
[[ 0.03144226  0.11875394  0.84321886  0.00658482]]
('unknown/out05060.png', array([2]))
[[  1.59301073e-03   3.53275915e-03   9.94750082e-01   1.24153681e-04]]
('unknown/out00118.png', array([2]))
[[  4.21493081e-03   1.27817830e-02   9.82554615e-01   4.48686827e-04]]
('unknown/out00078.png', array([2]))
[[ 0.0557854   0.14251819  0.78525859  0.01643789]]
('unknown/out00252.png', array([2]))
[[  3.2357

[[  5.16655017e-03   2.41395161e-02   9.70144570e-01   5.49336837e-04]]
('unknown/out07748.png', array([2]))
[[ 0.01018167  0.03917047  0.94916517  0.00148272]]
('unknown/out07990.png', array([2]))
[[  4.73656179e-03   1.61996279e-02   9.78528202e-01   5.35671657e-04]]
('unknown/out02368.png', array([2]))
[[ 0.33412096  0.22491199  0.2860055   0.15496151]]
('unknown/out06678.png', array([0]))
[[ 0.01186863  0.02801877  0.95835644  0.00175609]]
('unknown/out06320.png', array([2]))
[[  6.22502202e-03   2.73118597e-02   9.65741456e-01   7.21624645e-04]]
('unknown/out07176.png', array([2]))
[[  1.43446226e-03   6.87793735e-03   9.91573155e-01   1.14522300e-04]]
('unknown/out00802.png', array([2]))
[[  2.66196229e-03   1.20908488e-02   9.85013127e-01   2.34035964e-04]]
('unknown/out06042.png', array([2]))
[[  7.60198745e-04   6.85608992e-03   9.92341757e-01   4.19966345e-05]]
('unknown/out03648.png', array([2]))
[[ 0.02594074  0.08577469  0.88333976  0.00494488]]
('unknown/out00432.png', ar

[[  1.39623228e-03   2.01785564e-03   1.37222014e-05   9.96572256e-01]]
('yellow/out00452.png', array([3]))
[[  2.96128122e-03   2.62499135e-03   3.24582725e-05   9.94381368e-01]]
('yellow/out00446.png', array([3]))
[[  7.47805345e-04   2.85895076e-04   1.75605328e-06   9.98964548e-01]]
('yellow/out01878.png', array([3]))
[[  6.77399524e-03   2.16240808e-03   5.42924681e-05   9.91009355e-01]]
('yellow/out02828.png', array([3]))
[[  1.11705239e-03   1.02903316e-04   9.26063990e-07   9.98779118e-01]]
('yellow/out01886.png', array([3]))
[[ 0.04147659  0.06804386  0.00336534  0.88711423]]
('yellow/out08718.png', array([3]))
[[  3.55113000e-02   2.83207977e-03   1.96590627e-04   9.61460054e-01]]
('yellow/out08876.png', array([3]))
[[  5.46477130e-03   8.56006704e-03   1.38800489e-04   9.85836267e-01]]
('yellow/out00156.png', array([3]))
[[  1.51126394e-02   1.14183989e-03   4.80418130e-05   9.83697474e-01]]
('yellow/out01466.png', array([3]))
[[  8.92095082e-03   2.81722721e-04   8.20784135

In [10]:
print('Classification Report')
print(classification_report(test_generator.classes, pred_class_arr))

Classification Report
              precision    recall  f1-score   support

           0       0.98      0.93      0.96       130
           1       0.94      0.97      0.95       291
           2       0.96      0.97      0.97       281
           3       0.96      0.91      0.93        55

   micro avg       0.96      0.96      0.96       757
   macro avg       0.96      0.94      0.95       757
weighted avg       0.96      0.96      0.96       757

